## GeoCat experimental

In [24]:
import importlib

import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
from IPython.core.debugger import set_trace
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
importlib.reload(geocat)
importlib.reload(rec)
CITY="madison"



In [2]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})
df_checkin['date']=pd.to_datetime(df_checkin['date'])
# users=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
# num_users=len(users)
df_checkin.set_index("user_id",inplace=True)
df_checkin.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
0,0,2018-04-04 00:09:03,43.069425,-89.389567,"[coffee, cafes]",2064.0
1,0,2018-02-16 20:40:41,43.069425,-89.389567,"[coffee, cafes]",2064.0
2,0,2018-04-12 11:31:03,43.069425,-89.389567,"[coffee, cafes]",2064.0
3,0,2018-03-07 14:08:41,43.069425,-89.389567,"[coffee, cafes]",2064.0
1,0,2018-02-16 20:48:49,43.069425,-89.389567,"[coffee, cafes]",2064.0


In [3]:
df_poi=pd.read_csv("../data/poi/"+CITY+".csv",converters={'categories':string_to_array})
df_poi=df_poi.set_index("business_id")

In [4]:
df_poi.head(1)

,latitude,longitude,categories
business_id,,,
1744,43.075012,-89.447498,"[desserts, cafes, coffee]"


# Test and training set

In [5]:
# df_checkin=df_checkin.sort_values(['user_id','date'])

# ranks=df_checkin.groupby('user_id')['date'].rank(method='first').reset_index(drop=True)
# ranks.head()

# #ranks=df_review.groupby('user_id')['date'].rank(method='first')
# count=df_checkin.reset_index()['user_id'].map(df_checkin.groupby('user_id')['date'].apply(len))
# count.head()

# training_set_delimiter=(ranks/count)<0.72
# test_set_delimiter=np.logical_not(training_set_delimiter)
# training_set_delimiter

# df_checkin_train=df_checkin.reset_index()[training_set_delimiter]
# df_checkin_train.count()

# df_checkin_test=df_checkin.reset_index()[test_set_delimiter]
# df_checkin_test.count()

# users=df_checkin_train['user_id'].drop_duplicates().reset_index(drop=True)



In [6]:
df_checkin_train=pd.read_csv("../data/checkin/train/"+CITY+".csv",converters={'categories':string_to_array})
df_checkin_test=pd.read_csv("../data/checkin/test/"+CITY+".csv",converters={'categories':string_to_array})


In [7]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [8]:
users=df_checkin_train['user_id'].drop_duplicates().reset_index(drop=True)
num_users=len(users)
users.count()

1022

In [9]:
# df_checkin_train[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/train/"+CITY+".csv",index=False,header=False)

# df_checkin_test[['user_id','business_id','date']].groupby(['user_id','business_id']).count().reset_index().to_csv("../data/checkin/test/"+CITY+".csv",index=False,header=False)

In [10]:
df_checkin_train=df_checkin_train.set_index('user_id')
df_checkin_test=df_checkin_test.set_index('user_id')

In [11]:
series_poi_checkin=df_checkin_train.reset_index().groupby('business_id').count()['user_id']
series_poi_checkin.name="checkin"
series_poi_checkin.head(2)

business_id
0     2
1    58
Name: checkin, dtype: int64

### load poi neighbor

In [12]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")

In [54]:
N=80
K=20
VERY_SMALL_VALUE=-100
i=0

# tmp_rec_list=rec.mostpopular(df_checkin,K)
# tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
# print(tmp_rec_list)
dict_rec_list=dict()
for user_id in users:
    print(user_id)
    if user_id ==0:
        continue
    #user_id="scaxsfOynKUZEijn_UCTkg"
    #print(user_id)
    # User visits
    df_user_checkin=df_checkin_train.loc[[user_id]].reset_index(drop=True)
    
    ####print(df_user_checkin['business_id'])
    
    list_user_pois=set(df_user_checkin['business_id'].drop_duplicates().values)
    
    
    tmp_rec_list=rec.mostpopular(df_checkin_train.reset_index(),N,num_users,list_user_pois)
    
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_poi.reset_index(),on='business_id').drop_duplicates(subset=df_poi.reset_index().columns.difference(['categories'])).reset_index(drop=True)
    #print(tmp_rec_list)
    
    business_cover=list()
    pois_id=set()
    for business_id in df_user_checkin['business_id'].reset_index(drop=True):
        business_cover.append(0)
        pois_id.add(business_id)
    pois_id=list(pois_id)
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.75 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.5 # lambda, geo vs cat
    current_proportionality=0
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
#     relevant_categories_user=geocat.relevant_categories_to_the_user(df_user_checkin)

    # REPLICATE WRONG CODE
    series_user_checkin=df_user_checkin.groupby('business_id').count()['date']
    series_user_checkin.name="checkin"

    
    # This code is to replicate the error in original geocat
    ##mean_poi_visits=(series_poi_checkin[pois_id]*series_user_checkin).sum()/series_user_checkin.sum()
    mean_poi_visits=(series_poi_checkin[pois_id]).mean()

    # using entry from java
    #####mean_poi_visits=pd.read_csv("~/checkinjava.csv").set_index("business_id")['checkin'].mean()
    # Relevant categories

    relevant_categories = set()
    print(mean_poi_visits)
    #pd.read_csv("~/checkinjava.csv").set_index("business_id")['checkin']
    ####for poi_id,checkins in series_poi_checkin[business_cover.keys()].iteritems():
    
    for poi_id,checkins in series_poi_checkin[pois_id].iteritems():
        # Check if poi is relevant
        if checkins >= mean_poi_visits:
            # add relevant categories
#             print(poi_id)
#             print(df_poi.loc[poi_id]['categories'])
            for category in df_poi.loc[poi_id]['categories']:
                relevant_categories.add(category)
        
        

    print("Number of relevant categories:",len(relevant_categories))#,"Number of relevant cat dupl:",cc)
    # REPLICATE WRONG CODE
    #for i in relevant_categories:
   #     print(dict_alias_title[i])
    #print(relevant_categories)

    
    # post-processing for diversification
    for i in range(K):
#         start = timeit.default_timer()
        #num_current_candidates=len(tmp_rec_list)
       # set_trace()
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        rec_list_categories=geocat.category_in_rec_list(tmp_rec_list)
        print(i)
        
        for index,poi_candidate in tmp_rec_list.iterrows():
            candidate_neighbors=df_poi_neighbor.loc[poi_candidate.business_id].neighbors
            
            
            poi_neighbors=df_poi.loc[set(candidate_neighbors) & list_user_pois]
            #print(len(poi_neighbors))
            
            objective_value=geocat.objective_ILD_GC_PR(poi_candidate,df_user_checkin,rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight,dict_alias_title,undirected_category_tree,\
                                                      relevant_categories,rec_list_categories,poi_neighbors)
            #print(poi_candidate.business_id,objective_value)
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate 
        
#         stop = timeit.default_timer()
#         print('Time:', stop - start)
        
        if poi_to_insert is not None:
            print(poi_to_insert.business_id,max_objective_value)
            candidate_neighbors=df_poi_neighbor.loc[poi_to_insert.business_id].neighbors
            poi_neighbors=df_poi.loc[set(candidate_neighbors) & list_user_pois]
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=geocat.update_geo_cov(poi_to_insert,df_user_checkin,K,business_cover,poi_neighbors)
            #print(current_proportionality)
            
    
    dict_rec_list[user_id]=rec_list[['business_id','score']]
    print(rec_list[['business_id','score']].sort_values(by="score",ascending=False).reset_index(drop=True))
    
    i+=1
    if i % num_users==0:
        print(i)
    break

0
1
68.05
Number of relevant categories: 25
0
2326 1.0 0.16 0.1022922983626442
10 1.0 0.08 0.010283505154639161
343 1.0 0.12 0.10136082474226826
361 1.0 0.0 0.030927835051546393
2332 1.0 0.08 0.1038800374882849
1338 1.0 0.04 0.1022922983626442
9 1.0 0.08 0.010283505154639161
1040 1.0 0.08 0.10452783505154639
2675 1.0 0.12 0.09231222385861582
2658 1.0 0.08 0.09740206185567035
3006 1.0 0.08 0.030927835051546393
2018 1.0 0.0 0.09608247422680405
1688 1.0 0.0 0.010283505154639161
68 1.0 0.12 0.010283505154639161
682 1.0 0.08 0.010283505154639161
2688 1.0 0.08 0.05154639175257736
1017 1.0 0.04 0.030927835051546393
2692 1.0 0.04 0.030927835051546393
2313 1.0 0.0 0.010283505154639161
3029 1.0 0.08 0.1022922983626442
1363 1.0 0.08 0.010283505154639161
1048 1.0 0.0 0.10302767227346687
2717 1.0 0.16 0.010283505154639161
2010 1.0 0.08 0.010283505154639161
1662 1.0 0.04 0.05154639175257736
2666 1.0 0.04 0.10334020618556705
694 1.0 0.0 0.1022922983626442
2392 1.0 0.04 0.030927835051546393
1388 1.0 0

2022 0.0 0.28 0
42 0.6666666666666667 0.28 0
1683 0.6666666666666667 0.32 0
676 0.0 0.28 0.05602813826561581
1666 0.0 0.32 0.05894365069739238
1073 0.0 0.32 0
2746 0.0 0.36 0
1715 0.0 0.28 0
374 0.0 0.32 0
2371 0.0 0.28 0.07826495396659128
1371 0.6666666666666667 0.32 0.057867798665858006
1723 0.6666666666666667 0.32 0.020618556701030855
19 0.8 0.28 0
1418 0.0 0.28 0
2317 0.0 0.28 0
2701 0.0 0.36 0.07385920471281271
1028 0.0 0.28 0
2014 0.0 0.28 0.07385920471281271
2359 0.6666666666666667 0.32 0.059381443298969105
1678 0.8 0.28 0
348 0.6666666666666667 0.32 0
693 0.0 0.28 0.05602813826561581
2043 0.6666666666666667 0.28 0.07521649484536086
725 0.0 0.28 0
60 0.0 0.32 0.0645511218920557
1778 0.0 0.28 0.020618556701030855
2109 0.0 0.28 0.06927835051546383
2332 0.12272881654861706
4
10 0.0 0.32 0
361 0.6666666666666667 0.28 0.020618556701030966
1338 0.0 0.28 0.045866607861514086
9 0.0 0.28 0
2675 0.0 0.28 0.06923207926094543
2658 0.0 0.28 0.07206388003748831
3006 0.6666666666666667 0.36 0.

1108 0.6666666666666667 0.32 0.04286609987950185
375 0.0 0.4 0.04430464193174943
1046 0.0 0.32 0.04990461742960217
2067 0.0 0.32 0
3013 0.0 0.32 0
2022 0.0 0.32 0
42 0.6666666666666667 0.32 0
1683 0.0 0.32 0
676 0.0 0.32 0.04589238105739024
1666 0.0 0.36 0.05249144826065377
1073 0.0 0.36 0
2746 0.0 0.36 0
1715 0.0 0.32 0
374 0.0 0.36 0
2371 0.0 0.32 0.04982689483734193
1371 0.0 0.36 0.05789357186173427
1723 0.6666666666666667 0.36 2.5773195876266364e-05
19 0.8 0.32 0
1418 0.0 0.32 0
2317 0.0 0.32 0
2701 0.0 0.4 0.04286609987950185
1028 0.0 0.32 0
2014 0.0 0.32 0.04286609987950185
2359 0.6666666666666667 0.36 0.05940721649484537
1678 0.8 0.32 0
348 0.6666666666666667 0.36 0
693 0.0 0.32 0.04589238105739024
2043 0.0 0.32 0.07524226804123713
725 0.0 0.32 0
60 0.0 0.36 0.0477821434478195
1778 0.0 0.32 2.5773195876266364e-05
2109 0.0 0.32 0.0693041237113402
1338 0.11877073414588801
8
9 0.0 0.32 0
2675 0.0 0.32 0.042840326683625474
2658 0.0 0.32 0.04831130271790074
3006 0.6666666666666667 0.

1388 0.0 0.52 0
371 0.0 0.48 0
2319 0.6666666666666667 0.48 0
16 0.0 0.48 0
2724 0.0 0.52 0.00046686770928305865
743 0.0 0.48 0.0210140577728124
704 0.0 0.52 0
2758 0.0 0.48 0
2344 0.0 0.48 0.018267992197053107
1050 0.0 0.52 0
373 0.0 0.52 0
3035 0.0 0.48 0
1712 0.0 0.48 0.020618556701031077
2357 0.0 0.48 0
2049 0.0 0.48 0.019188882849859867
2366 0.0 0.48 0
2006 0.0 0.48 0.023676584884750662
3015 0.0 0.48 0
2672 0.0 0.48 0
1381 0.0 0.48 0
1108 0.6666666666666667 0.48 0.0006504340182809099
375 0.0 0.56 0.03130477188063607
1046 0.0 0.48 0.022090916340544386
2067 0.0 0.48 0
3013 0.0 0.48 0
2022 0.0 0.48 0
42 0.6666666666666667 0.48 0
1683 0.0 0.48 0
676 0.0 0.48 0.03189450355587409
1666 0.0 0.52 0.04268520205082971
1073 0.0 0.48 0
2746 0.0 0.52 0
1715 0.0 0.48 0
374 0.0 0.52 0
2371 0.0 0.48 0.01151531681727258
1371 0.0 0.52 0.054411757571433794
1723 0.6666666666666667 0.52 0
19 0.8 0.48 0
1418 0.0 0.48 0
2317 0.0 0.48 0
2701 0.0 0.52 0.0006504340182809099
1028 0.0 0.48 0
2014 0.0 0.48 0.0

KeyboardInterrupt: 

In [50]:
1-1/1

0.17695473251028815

In [14]:
pd.read_csv("~/teste.csv",header=None)

,0
0,OSXb-DOITpDR7nm8wjrZ2w
1,U7G4VBrra439LgGBvV7dcg
2,_q0ci_oTC-QccjFKVc6CeA
3,qAzz1OYoGjwcSdwzu6FgTg
4,Iq1P8S3--gqd50xoQJxpqA
5,kQ81FVLQWKa9xEqZGQV_ng
6,hySu4oYN7R3sSohuSeEsTg
7,VfWX3UCKvUnVktdOuR8TwA
8,eZ1u1mwjUqb62v9dSiqZaQ
9,OlA7h9oOttqTALudvEUzTg


In [15]:
dict_alias_title['']

KeyError: ''

In [ ]:
pd.merge(pd.read_csv("~/a.txt",header=None),
pd.read_csv("~/b.txt",header=None),how='inner')